# $Z^0$ decays: finding the $Z^0$ boson mass

In [ ]:
import uproot
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
f = uproot.open("https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/1largeRjet1lep/MC/mc_361106.Zee.1largeRjet1lep.root")

In [ ]:
events = f["mini"]

In [ ]:
import hist
from hist import Hist

In [ ]:
Z0_hist = Hist(hist.axis.Regular(30, 40, 100, name="Mass [GeV]"))

## Cuts

Uproot

In [ ]:
def inv_mass(pt1,pt2,eta1,eta2,phi1,phi2):
    return np.sqrt(2*pt1*pt2 * (np.cosh(eta1 - eta2) - np.cos(phi1 - phi2)))

In [ ]:
sel_events = events.arrays(["lep_pt", "lep_eta", "lep_phi", "lep_E","lep_charge", "lep_type", "lep_n"])

for event in sel_events:
    # Cut #1: At least 2 leptons in the event. lep_n  is the number of them.
    
    lep_n = event["lep_n"]    
    if lep_n >= 2:
        
        # Cut #2: Leptons with opposite charge.
        #We have a list of charges, each corresponding to a lepton: lep_charges.
        #Clearly, we can't let these be equal
        lep_charge = event["lep_charge"]
        if (lep_charge[0] != lep_charge[1]):
            
            # Cut #3: Leptons of the same family (2 electrons or 2 muons).
            # lep_type gives back a number, which is a code for what kind of lepton it is.
            lep_type = event["lep_type"]
            if ( lep_type[0] == lep_type[1]):
                
                # By now we should only have paricles that look right remaining.
                
                # Let's set the components of a TLorentzVector for each lepton.
                # Notice that the energy and momenta are given in MeV!
                
                lep_pt = event["lep_pt"]
                lep_eta = event["lep_eta"]
                lep_phi = event["lep_phi"]
                lep_E = event["lep_E"]
                
                Z0_boson = inv_mass(lep_pt[0],lep_pt[1],lep_eta[0],lep_eta[1],lep_phi[0],lep_phi[1])
                
                #leadLepton.SetPtEtaPhiE(lep_pt[0]/1000., lep_eta[0], lep_phi[0], lep_E[0]/1000.)
                #trailLepton.SetPtEtaPhiE(lep_pt[1]/1000., lep_eta[1], lep_phi[1], lep_E[1]/1000.)
                
                
                
                # Now, reconstruct the Z0 boson Lorentz vector! 
                # Remember, we can add them just like normal vectors.
                #Z0_boson = leadLepton + trailLepton
                
                # Put this particular value into the histogram.
                Z0_hist.fill(Z0_boson)

In [ ]:
Z0_hist.plot()
plt.title("Invariant mass of the $Z^0$ boson")
plt.show()

## .csv file

#### Getting data

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
f = pd.read_csv('DoubleMuRun2011A.csv')

In [ ]:
len(f)

In [ ]:
f.head(5)

#### Plotting px

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import hist
from hist import Hist

In [ ]:
px_hist = Hist(hist.axis.Regular(100, -20, 20, name="px"))

In [ ]:
px1 = f['px1']
#plt.hist(px1, bins=100, range=(-20.,20.))
px_hist.fill(px1)
px_hist.plot(histtype = "fill")
plt.show()

In [ ]:
px_hist.fill(px1)
px_hist.plot(histtype = "fill")
plt.xlabel('x-component of momentum [GeV]')
plt.ylabel('Number of events')
plt.title('Histogram of px for muon 1. \n')

plt.show()

#### Plotting invariant mass

In [ ]:
invm = f['M']
nbins = 500
invm_hist = Hist(hist.axis.Regular(nbins, 0.5, 120, name="Invariant mass [GeV]"))
invm_hist.fill(invm)
invm_hist.plot(histtype = "fill")

plt.show()

In [ ]:
invm_hist.fill(invm)
invm_hist.plot(histtype = "fill")
plt.yscale('log')
plt.xscale('log')

plt.show()

In [ ]:
# You need the Mass of the Muon to calculate the energy
muMass = 0.105658

# Momentum squared for the two individual muons
p1_squared = (f["px1"])**2 + (f["py1"])**2 + (f["pz1"])**2
p2_squared = (f["px2"])**2 + (f["py2"])**2 + (f["pz2"])**2

# Energy of the two individual muons
e1 = np.sqrt(p1_squared + (muMass*muMass))
e2 = np.sqrt(p2_squared + (muMass*muMass))

# Total Energy of the two muons 
epair = e1 + e2 

# Momentum squared of the muon pair vector (p1+p2) - remember to add the vectors before squaring
ptpair_squared = (f["px1"] + f["px2"])**2 + (f["py1"] + f["py2"])**2 + (f["pz1"] + f["pz2"])**2

# Invariant mass of the muon pair, save this in  variable called "invariant_mass"
invariant_mass = np.sqrt(epair**2 - ptpair_squared)

In [ ]:
invar_hist = Hist(hist.axis.Regular(100, 60, 120, name="Invariant mass [GeV]"))
invar_hist.fill(invariant_mass)
invar_hist.plot(histtype = "fill")
plt.show()

## Higgs to 4 leptons

In [ ]:
# Here we load the data for the different final sets of particles
f_2e2mu_2011 = pd.read_csv('2e2mu_2011.csv')
f_2e2mu_2012 = pd.read_csv('2e2mu_2012.csv')
f_4e_2011 = pd.read_csv('4e_2011.csv')
f_4e_2012 = pd.read_csv('4e_2012.csv')
f_4mu_2011 = pd.read_csv('4mu_2011.csv')
f_4mu_2012 = pd.read_csv('4mu_2012.csv')

# Here we concatenate the 6 datasets into one called "f2"
f2 = pd.concat([f_2e2mu_2011, f_2e2mu_2012, f_4e_2011, f_4e_2012, f_4mu_2011, f_4mu_2012], axis=0, ignore_index=True)

In [ ]:
# Total Energy of the four leptons
E = f2["E1"] + f2["E2"] + f2["E3"] + f2["E4"]

#Total momentum in the x direction of the four leptons
px = f2["px1"] + f2["px2"] + f2["px3"] + f2["px4"]

#Total momentum in the y direction of the four leptons
py = f2["py1"] + f2["py2"] + f2["py3"] + f2["py4"]

#Total momentum in the z direction of the four leptons
pz = f2["pz1"] + f2["px2"] + f2["pz3"] + f2["pz4"]

# Now calculate the invariant mass using Equation (2) above and assign it to a variable called 'invariant_mass_2e2mu'
invariant_mass_2e2mu = np.sqrt(E**2 - px**2 - py**2 - pz**2)

In [ ]:
leps_hist = Hist(hist.axis.Regular(60, 45, 180, name="Invariant mass [GeV]"))
leps_hist.fill(invariant_mass_2e2mu)
leps_hist.plot(histtype = "fill")

plt.xlabel('Invariant mass [GeV]')
plt.ylabel('Number of events')
plt.title('Histogram of invariant mass values of four leptons. \n')

plt.ylim(0,9)
plt.arrow(70, 7.5, 15, -4.2,length_includes_head=True, width=0.2, fc='r', ec='r')
plt.text(60, 8, 'Z Boson', fontsize=12)
plt.arrow(125, 7.5, -2, -3.4,length_includes_head=True, width=0.5, fc='r', ec='r')
plt.text(110, 8, 'Higgs Boson', fontsize=12)

plt.show()